Setting runtime environment

In [ ]:
RUN_FROM = 'LOCAL'
# RUN_FROM = 'COLAB'

if RUN_FROM == 'LOCAL':
    from os.path import expanduser
    HOME = expanduser("~")
elif RUN_FROM == 'COLAB':
    !pip install hanziconv
    from google.colab import drive
    drive.mount('/content/drive')
    %cd /content/drive/My Drive/projects/lm
    HOME = '/content/drive/My Drive'

Import modules

In [ ]:
import os
from data import Dataset
from model import Model
from routines import lr_range_test, train_and_evaluate
from utils import data_utils_py3

Setting parameters

In [ ]:
if RUN_FROM == 'LOCAL':
    train_dir = 'pretrain_zh_small'
elif RUN_FROM == 'COLAB':
    train_dir = HOME+'/Models/pretrain_zh_small'

data_dir = HOME+"/Data/text_zh"
char_vocab_file = HOME+"/Data/Vocab/vocab_zh"
char_embedding_files = HOME+"/Data/Vocab/zh_char_300_nlpcc.txt"
char_vocab = data_utils_py3.Vocab(
    char_vocab_file,
    embedding_files=char_embedding_files)
dataset = Dataset(data_dir, char_vocab)

model_config = {
    'char_vocab_size': char_vocab.size(),
    'char_vocab_dim': 300,
    'char_vocab_emb': char_vocab.embedding_init,
    'layer_size': 256,
    'num_layers': 3,
}
model = Model(model_config)

run_config = {
    'batch_size': 32,
    'max_lr': 5e-5,
    'max_train_steps': 200000,
    'pct_start': 0.3,
    'dropout': 0.1,
    'wd': .0,
    'max_char_length': 8,
    'data': [{'is_target': True, 'max_token_length': 8, 'min_seq_length': 2, 'max_seq_length': 256},],
}

LR range test and plot the curve

In [ ]:
lr_range_test(dataset, model, run_config)

Start train and eval loop

In [ ]:
train_and_evaluate(
    dataset, model, run_config, train_dir, eval_every=10000)